# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
A. Winter  ->  A. Winter  |  ['A. Winter']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
A. Winter  ->  A. Winter  |  ['A. Winter']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang 

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
F. Briegel  ->  F. Briegel  |  ['F. Briegel']
L. Mohr  ->  L. Mohr  |  ['L. Mohr']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
Arxiv has 105 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.18616


extracting tarball to tmp_2504.18616...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


...

extracting tarball to tmp_2504.20036...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2504.20036/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_dvphi_prop' from 'tmp_2504.20036/table_dvphi_prop.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authorlist' from 'tmp_2504.20036/authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Flock  ->  M. Flock  |  ['M. Flock']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']


Found 91 bibliographic references in tmp_2504.20036/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18616-b31b1b.svg)](https://arxiv.org/abs/2504.18616) | **Insights on Metal Enrichment and Environmental Effect at $z\approx5-7$ with JWST ASPIRE/EIGER and Chemical Evolution Model**  |
|| Z. Li, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *19 pages, 9 figures. Comments are welcome*|
|**Abstract**|            We present the mass-metallicity relation (MZR) for a parent sample of 604 galaxies at $z=5.34-6.94$ with [\text{O}~\textsc{iii}] doublets detected, using the deep JWST/NIRCam wide field slitless spectroscopic (WFSS) observations in 26 quasar fields. The sample incorporates the full observations of 25 quasar fields from JWST Cycle 1 GO program ASPIRE and the quasar SDSS J0100+2802 from JWST EIGER program. We identify 204 galaxies residing in overdense structures using friends-of-friends (FoF) algorithm. We estimate the electron temperature of $2.0^{+0.3}_{-0.4}\times10^4$ K from the Hg and $[\text{O}~\textsc{iii}]_{4363}$ lines in the stacked spectrum, indicating a metal-poor sample with median gas phase metallicity 12+$\log(\mathrm{O/H})=7.64^{+0.23}_{-0.11}$. With the most up-to-date strong line calibration based on NIRSpec observations, we find that the MZR shows a metal enhancement of $\sim0.2$ dex at high mass end in overdense environments. However, compared to the local Fundamental Metallicity Relation (FMR), our galaxy sample at $z>5$ shows a metal deficiency of $\sim0.2$ dex relative to FMR predictions. We explain the observed trend of FMR with a simple analytical model, and we favor dilution from intense gas accretion over outflow to explain the metallicity properties at $z>5$. Those high redshift galaxies are likely in a rapid gas accretion phase when their metal and gas contents are in a non-equilibrium state. According to model predictions, the protocluster members are closer to the gas equilibrium state than field galaxies and thus have higher metallicity and are closer to the local FMR. Our results suggest that the accelerated star formation during protocluster assembly likely plays a key role in shaping the observed MZR and FMR, indicating a potentially earlier onset of metal enrichment in overdense environments at $z\approx5-7$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18643-b31b1b.svg)](https://arxiv.org/abs/2504.18643) | **exoALMA VII: Benchmarking Hydrodynamics and Radiative Transfer Codes**  |
|| J. Bae, et al. -- incl., <mark>M. Flock</mark>, <mark>M. Benisty</mark>, <mark>I. Hammond</mark>, <mark>A. Winter</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *16 pages, 13 figures, Accepted for publication in The Astrophysical Journal Letters*|
|**Abstract**|            Forward modeling is often used to interpret substructures observed in protoplanetary disks. To ensure the robustness and consistency of the current forward modeling approach from the community, we conducted a systematic comparison of various hydrodynamics and radiative transfer codes. Using four grid-based hydrodynamics codes (FARGO3D, Idefix, Athena++, PLUTO) and a smoothed particle hydrodynamics code (Phantom), we simulated a protoplanetary disk with an embedded giant planet. We then used two radiative transfer codes (mcfost, RADMC-3D) to calculate disk temperatures and create synthetic 12CO cubes. Finally, we retrieved the location of the planet from the synthetic cubes using DISCMINER. We found strong consistency between the hydrodynamics codes, particularly in the density and velocity perturbations associated with planet-driven spirals. We also found a good agreement between the two radiative transfer codes: the disk temperature in mcfost and RADMC-3D models agrees within $\lesssim 3~\%$ everywhere in the domain. In synthetic $^{12}$CO channel maps, this results in brightness temperature differences within $\pm1.5$ K in all our models. This good agreement ensures consistent retrieval of planet's radial/azimuthal location with only a few % of scatter, with velocity perturbations varying $\lesssim 20~\%$ among the models. Notably, while the planet-opened gap is shallower in the Phantom simulation, we found that this does not impact the planet location retrieval. In summary, our results demonstrate that any combination of the tested hydrodynamics and radiative transfer codes can be used to reliably model and interpret planet-driven kinematic perturbations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18688-b31b1b.svg)](https://arxiv.org/abs/2504.18688) | **exoALMA I. Science Goals, Project Design and Data Products**  |
|| R. Teague, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters*|
|**Abstract**|            Planet formation is a hugely dynamic process requiring the transport, concentration and assimilation of gas and dust to form the first planetesimals and cores. With access to extremely high spatial and spectral resolution observations at unprecedented sensitivities, it is now possible to probe the planet forming environment in detail. To this end, the exoALMA Large Program targeted fifteen large protoplanetary disks ranging between ${\sim}1\arcsec$ and ${\sim}7\arcsec$ in radius, and mapped the gas and dust distributions. $^{12}$CO J=3-2, $^{13}$CO J=3-2 and CS J=7-6 molecular emission was imaged at high angular (${\sim}~0\farcs15$) and spectral (${\sim}~100~{\rm m\,s^{-1}}$) resolution, achieving a surface brightness temperature sensitivity of ${\sim}1.5$~K over a single channel, while the 330~GHz continuum emission was imaged at 90~mas resolution and achieved a point source sensitivity of ${\sim}\,40~\mu{\rm Jy~beam^{-1}}$. These observations constitute some of the deepest observations of protoplanetary disks to date. Extensive substructure was found in all but one disk, traced by both dust continuum and molecular line emission. In addition, the molecular emission allowed for the velocity structure of the disks to be mapped with excellent precision (uncertainties on the order of $10~{\rm m\,s^{-1}}$), revealing a variety of kinematic perturbations across all sources. From this sample it is clear that, when observed in detail, all disks appear to exhibit physical and dynamical substructure indicative of on-going dynamical processing due to young, embedded planets, large-scale, (magneto-)hydrodynamical instabilities or winds.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18717-b31b1b.svg)](https://arxiv.org/abs/2504.18717) | **exoALMA. X. channel maps reveal complex $^{12}$CO abundance distributions and a variety of kinematic structures with evidence for embedded planets**  |
|| C. Pinte, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *16 pages, 8 figures, accepted for publication in the ApJL exoALMA focus issue*|
|**Abstract**|            We analyze the $^{12}$CO $J=3-2$ data cubes of the disks in the exoALMA program. 13/15 disks reveal a variety of kinematic substructures in individual channels: large-scale arcs or spiral arms, localized velocity kinks, and/or multiple faints arcs that appear like filamentary structures on the disk surface. We find kinematic signatures that are consistent with planet wakes in six disks: AA Tau, SY Cha, J1842, J1615, LkCa 15 and HD 143006. Comparison with hydrodynamical and radiative transfer simulations suggests planets with orbital radii between 80 and 310\,au and masses between 1 and 5 M$_\mathrm{Jup}$. Additional kinematic substructures limit our ability to place tight constraints on the planet masses. When the inclination is favorable to separate the upper and lower surfaces (near 45$^\mathrm{o}$, i.e. in 7/15 disks), we always detect the vertical CO snowline and find that the $^{12}$CO freeze-out is partial in the disk midplane, with a depletion factor of $\approx 10^{-3}$ - $10^{-2}$ compared to the warm molecular layer. In these same seven disks, we also systematically detect evidence of CO desorption in the outer regions.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.20036-b31b1b.svg)](https://arxiv.org/abs/2504.20036) | **exoALMA. VI. Rotating under Pressure: Rotation curves, azimuthal velocity substructures, and pressure variations**  |
|| J. Stadler, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters. 37 pages, 19 figures*|
|**Abstract**|            The bulk motion of the gas in protoplanetary disks around newborn stars is nearly Keplerian. By leveraging the high angular and spectral resolution of ALMA, we can detect small-scale velocity perturbations in molecular line observations caused by local gas pressure variations in the disk, possibly induced by embedded protoplanets. This paper presents the azimuthally averaged rotational velocity and its deviations from Keplerian rotation ($\delta\upsilon_{\phi}$) for the exoALMA sample, as measured in the $^{12}$CO and $^{13}$CO emission lines. The rotation signatures show evidence for vertically stratified disks, in which $^{13}$CO rotates faster than $^{12}$CO due to a distinct thermal gas pressure gradient at their emitting heights. We find $\delta\upsilon_{\phi}$-substructures in the sample on both small ($\sim$10 au) and large ($\sim$100 au) radial scales, reaching deviations up to 15% from background Keplerian velocity in the most extreme cases. More than 75% of the rings and 80% of the gaps in the dust continuum emission resolved in $\delta\upsilon_{\phi}$ are co-located with gas pressure maxima and minima, respectively. Additionally, gas pressure substructures are observed far beyond the dust continuum emission. For the first time, we determined the gas pressure derivative at the midplane from observations and found it to align well with the dust substructures within the given uncertainties. Based on our findings, we conclude that gas pressure variations are likely the dominant mechanism for ring and gap formation in the dust continuum.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18820-b31b1b.svg)](https://arxiv.org/abs/2504.18820) | **Potential-Driven Metal Cycling: JADES Census of Gas-Phase Metallicity for galaxies at 1 < z < 7**  |
|| C. Jia, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *12 pages, 7 figures*|
|**Abstract**|            The gravitational potential is established as a critical determinant of gas-phase metallicity (12+log(O/H)) in low-redshift galaxies, whereas its influence remains unconfirmed at high redshifts. We investigate the correlation between gas-phase metallicity and effective radius ($R_{\rm e}$) for a sample of galaxies with redshifts ranging from 1 to 7, drawn from JADES (JWST Advanced Deep Extragalactic Survey) Data Release 3. We calculate the metallicities using four strong-line methods: ${\rm N2S2H\alpha}$, ${\rm R23}$, ${\rm N2}$, and ${\rm O3N2}$, respectively. After taking out the evolution of size, we find that the offsets of mass-size relation ($\Delta \log R_{\rm e}$) are significantly negatively correlated with the offset of mass-metallicity relation ($\Delta \log({\rm O/H})$) for the four metallicity tracers. Regardless of the metallicity tracer used, we obtain Spearman rank $p-$values much less than 0.01, rejecting the null hypothesis that the observed correlation is statistically nonsignificant and attributable to random chance. This is also true for galaxies with $z>3$, with $p-$values less than 0.05 for the four metallicity tracers. We for the first time find evidence of size playing a key role in determining gas-phase metallicity towards cosmic dawn, suggesting that the gravitational potential influences their material-exchange processes with the surrounding environment at very early universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18935-b31b1b.svg)](https://arxiv.org/abs/2504.18935) | **Superradiant dark matter production from primordial black holes: Impact of multiple modes and gravitational wave emission**  |
|| N. Jia, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *29 pages, 3 figures*|
|**Abstract**|            Rotating primordial black holes (PBHs) in the early universe can emit particles through superradiance, a process particularly efficient when the particle's Compton wavelength is comparable to the PBH's gravitational radius. Superradiance leads to an exponential growth of particle occupation numbers in gravitationally bound states. We present an analysis of heavy bosonic dark matter (DM) production through three gravitational mechanisms: Hawking radiation, superradiant instabilities, and ultraviolet (UV) freeze-in. We consider PBHs that evaporate before Big Bang Nucleosynthesis (BBN). For both scalar and vector DM, our analysis incorporates the evolution of a second superradiant mode. We demonstrate that the growth of a second superradiant mode causes the decay of the first mode, and thus the second mode cannot further enhance the DM abundance beyond that already achieved by the first mode. Our study also reveals that while superradiance generally enhances DM production, gravitational wave (GW) emission from the superradiant cloud may significantly modify this picture. For scalar DM, GW emission reduces the parameter space where superradiance effectively augments relic abundance. For vector DM, rapid GW emission from the superradiant cloud may yield relic abundances below those achieved through Hawking radiation alone. These findings demonstrate that multiple-mode effect and GW emission play critical roles in modeling DM production from PBHs in the early universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18952-b31b1b.svg)](https://arxiv.org/abs/2504.18952) | **Insight-HXMT observations of the extremely bright GRB 221009A**  |
|| W.-L. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-29*|
|*Comments*| *11 pages, 15 figures. Accepted for publication in ApJ*|
|**Abstract**|            The Hard X-ray Modulation Telescope (\insight) detected GRB 221009A, the brightest gamma-ray burst observed to date, with all its three telescopes, i.e. High Energy telescope (HE, 20-250 keV), Medium Energy telescope (ME, 5-30 keV), and Low Energy telescope (LE, 1-10 keV). Here we present the detailed observation results of all three telescopes of \insight~ on the prompt emission of GRB 221009A. After dead-time and data saturation correction, we recovered the light curves of HE, ME and LE telescopes and find that they generally track the GECAM-C low gain light curves that are free of data saturation issues. Particularly, the ME light curve matches the GECAM-C light curve in low gain mode above 400 keV, while the LE light curve is more consistent with the GECAM-C above 1.5 MeV. Based on simulation, we find that the signals recorded by the ME and LE are actually caused by the secondary particles produced by the interaction between GRB gamma-ray photons and the material of the satellite. Interestingly, the consistency between ME and LE light curves and GECAM-C demonstrates that ME and LE data could be used to characterize the GRB properties. Espeically, the high time resolution light curve of ME allowed us, for the first time, to calculate the minimum variability timescale (MVT = 0.10 s) of the main burst episode of GRB 221009A.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.18616/./SFR_Z_combined_plot.png', 'tmp_2504.18616/./MZR_vary_time.png', 'tmp_2504.18616/./SFR_Z_vary_time.png', 'tmp_2504.18616/./ASPIRE_MZR_obs.png']
copying  tmp_2504.18616/./SFR_Z_combined_plot.png to _build/html/
copying  tmp_2504.18616/./MZR_vary_time.png to _build/html/
copying  tmp_2504.18616/./SFR_Z_vary_time.png to _build/html/
copying  tmp_2504.18616/./ASPIRE_MZR_obs.png to _build/html/
exported in  _build/html/2504.18616.md
    + _build/html/tmp_2504.18616/./SFR_Z_combined_plot.png
    + _build/html/tmp_2504.18616/./MZR_vary_time.png
    + _build/html/tmp_2504.18616/./SFR_Z_vary_time.png
    + _build/html/tmp_2504.18616/./ASPIRE_MZR_obs.png
found figures ['tmp_2504.18643/././figures/dens2d_unstratified.png', 'tmp_2504.18643/././figures/dens1d_phi_unstratified.png', 'tmp_2504.18643/././figures/vtheta2d_unstratified.png']
copying  tmp_2504.18643/././figures/dens2d_unstratified.png to _build/html/
copying  tmp_2504.18643/././figures/dens1d_phi_unstratif

copying  tmp_2504.18717/./cont_scatt.png to _build/html/
exported in  _build/html/2504.18717.md
    + _build/html/tmp_2504.18717/./channels-1.png
    + _build/html/tmp_2504.18717/./channels-2.png
    + _build/html/tmp_2504.18717/./cont_scatt.png
found figures ['tmp_2504.18725/./Figures/gallery_single_sources_maintext.png', 'tmp_2504.18725/./Figures/gallery_single_sources_appendix1.png', 'tmp_2504.18725/./Figures/gallery_single_sources_appendix2.png']
copying  tmp_2504.18725/./Figures/gallery_single_sources_maintext.png to _build/html/
copying  tmp_2504.18725/./Figures/gallery_single_sources_appendix1.png to _build/html/
copying  tmp_2504.18725/./Figures/gallery_single_sources_appendix2.png to _build/html/
exported in  _build/html/2504.18725.md
    + _build/html/tmp_2504.18725/./Figures/gallery_single_sources_maintext.png
    + _build/html/tmp_2504.18725/./Figures/gallery_single_sources_appendix1.png
    + _build/html/tmp_2504.18725/./Figures/gallery_single_sources_appendix2.png
found f

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\iruv}{L_{\rm IR}/L_{\rm UV}}$
$\newcommand{\irx}{{\it IRX}}$
$\newcommand{\irxb}{{\it IRX}--\beta}$
$\newcommand{\iras}{{\it IRAS}}$
$\newcommand{\galex}{{\it GALEX}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\Mpc}{{\rm Mpc}}$
$\newcommand{\msun}{{M_\odot}}$
$\newcommand{\sfr}{{\rm M_\odot yr^{-1}}}$
$\newcommand{\lsun}{{\rm L_\odot}}$
$\newcommand{\hubunits}{{\rm km s^{-1} Mpc^{-1}}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\Gyr}{{\rm Gyr}}$
$\newcommand{\Hboiii}{{\rm H\beta+O[III]}}$
$\newcommand{\siiv}{{\rm Si{\sc IV}}}$
$\newcommand{\heii}{{\rm He{\sc ii}}}$
$\newcommand{\nii}{{\rm[N{\sc ii}]}}$
$\newcommand{\sii}{{\rm[S{\sc ii}]}}$
$\newcommand{\nv}{{\rm N{\sc V}}}$
$\newcommand{\mgii}{{\rm Mg{\sc ii}}}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\neiii}{{\rm Ne{\sc III}}}$
$\newcommand{\ciii}{{\rm C{\sc III}]}}$
$\newcommand{\civ}{{\rm C{\sc IV}}}$
$\newcommand{\cii}{{\rm[C{\sc II}]}}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Insights on Metal Enrichment and Environmental Effect at $z\approx5-7$ with JWST ASPIRE/EIGER and Chemical Evolution Model

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.18616-b31b1b.svg)](https://arxiv.org/abs/2504.18616)<mark>Appeared on: 2025-04-29</mark> -  _19 pages, 9 figures. Comments are welcome_

</div>
<div id="authors">

Z. Li, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present the mass-metallicity relation (MZR) for a parent sample of 604 galaxies at $z=5.34-6.94$ with $\OIII$ doublets detected, using the deep JWST/NIRCam wide field slitless spectroscopic (WFSS) observations in 26 quasar fields. The sample incorporates the full observations of 25 quasar fields from JWST Cycle 1 GO program ASPIRE and the quasar SDSS J0100+2802 from JWST EIGER program.    We identify 204 galaxies residing in overdense structures using friends-of-friends (FoF) algorithm. We estimate the electron temperature of $2.0^{+0.3}_{-0.4}\times10^4$ K from the $\Hg$ and $\OIII_{4363}$ lines in the stacked spectrum, indicating a metal-poor sample with median gas phase metallicity 12+ $\log(\mathrm{O/H})=7.64^{+0.23}_{-0.11}$ .    With the most up-to-date strong line calibration based on NIRSpec observations, we find that the MZR shows a metal enhancement of $\sim0.2$ dex at high mass end in overdense environments. However, compared to the local Fundamental Metallicity Relation (FMR), our galaxy sample at $z>5$ shows a metal deficiency of $\sim0.2$ dex relative to FMR predictions. We explain the observed trend of FMR with a simple analytical model, and we favor dilution from intense gas accretion over outflow to explain the metallicity properties at $z>5$ .    Those high redshift galaxies are likely in a rapid gas accretion phase when their metal and gas contents are in a non-equilibrium state. According to model predictions, the protocluster members are closer to the gas equilibrium state than field galaxies and thus have higher metallicity and closer to the local FMR.    Our results suggest that the accelerated star formation during protocluster assembly likely plays a key role in shaping the observed MZR and FMR, indicating a potentially earlier onset of metal enrichment in overdense environments at $z\approx5-7$ .

</div>

<div id="div_fig1">

<img src="tmp_2504.18616/./SFR_Z_combined_plot.png" alt="Fig11" width="100%"/>

**Figure 11. -** The predicted $Z-\text{SFR}$ relations for different model configurations. The first panel: Model 1, Incorporating inflow but no outflow, with $\epsilon_\text{SF}$ as the varying parameter. The second panel: Model 2, Incorporating outflow but no inflow, with $\epsilon_\text{SF}$ as the varying parameter. The third panel: Model 3, Incorporating both inflow and outflow, with $\eta$ as the varying parameter. The fourth panel: Model 4, Incorporating both inflow and outflow, with $\beta$ as the varying parameter. In all panels, the lines are color-coded by stellar mass, while different line styles indicate variations in model parameters. (*fig:FMR_predictions*)

</div>
<div id="div_fig2">

<img src="tmp_2504.18616/./MZR_vary_time.png" alt="Fig5.1" width="50%"/><img src="tmp_2504.18616/./SFR_Z_vary_time.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Top: The predicted MZR for galaxies with different $t_\text{obs}$. The dashed lines are MZRs color-coded by $t_\text{obs}$. This model assumes $\epsilon_\text{SF}=0.05$, $\eta_0=5$, and an energy-driven wind $\beta=2/3$. The diamond points are our observations the same as in Fig. \ref{fig:mzr}. Bottom: The predicted SFR$-$Z relation for galaxies is shown for different stellar masses and $t_\text{obs}$. The dashed lines, color-coded by $t_\text{obs}$, connect galaxies with the same observational time, while the solid black lines link galaxies of equal stellar mass. Different stellar masses are represented by different symbols. The red and blue diamond points are our observations in cluster and field environments. The asymmetric errors on SFR are due to uncertainties in dust correction.  (*fig:MZR_predictions*)

</div>
<div id="div_fig3">

<img src="tmp_2504.18616/./ASPIRE_MZR_obs.png" alt="Fig10" width="100%"/>

**Figure 10. -** The mass-metallicity relation for galaxies in protoclusters (red) and blank fields (blue), which are based on R3 calibration from \citetalias{Chakraborty_24}. The \citetalias{Sanders_24} calibration provides similar results, which are listed in Table. \ref{tab:mzr_info} for comparison.
    The results from other works in literature are shown with different colors, which are also listed in Table. \ref{tab:mzr_info} for reference. Higher redshift ($z>3$) measurements in literature are shown in solid lines, and lower redshift ($z\sim2-3$) measurements are shown in dashed lines.
     (*fig:mzr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.18616"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\fargo}{\texttt{FARGO3D}}$
$\newcommand{\inewcommandix}{\texttt{Inewcommandix}}$
$\newcommand{\athena}{\texttt{Athena++}}$
$\newcommand{\pluto}{\texttt{PLUTO}}$
$\newcommand{\Phantom}{\texttt{Phantom}}$
$\newcommand{\mcfost}{\texttt{mcfost}}$
$\newcommand{\radmc}{\texttt{RADMC-3D}}$
$\newcommand{\discminer}{\texttt{DISCMINER}}$</div>



<div id="title">

# exoALMA VII: Benchmarking Hydrodynamics and Radiative Transfer Codes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.18643-b31b1b.svg)](https://arxiv.org/abs/2504.18643)<mark>Appeared on: 2025-04-29</mark> -  _16 pages, 13 figures, Accepted for publication in The Astrophysical Journal Letters_

</div>
<div id="authors">

J. Bae, et al. -- incl., <mark>M. Flock</mark>, <mark>M. Benisty</mark>, <mark>I. Hammond</mark>, <mark>A. Winter</mark>

</div>
<div id="abstract">

**Abstract:** Forward modeling is often used to interpret substructures observed in protoplanetary disks. To ensure the robustness and consistency of the current forward modeling approach from the community, we conducted a systematic comparison of various hydrodynamics and radiative transfer codes. Using four grid-based hydrodynamics codes ( $\fargo$ , $\idefix$ , $\athena$ , $\pluto$ ) and a smoothed particle hydrodynamics code ( $\Phantom$ ), we simulated a protoplanetary disk with an embedded giant planet. We then used two radiative transfer codes ( $\mcfost$ , $\radmc$ ) to calculate disk temperatures and create synthetic $^{12}$ CO cubes. Finally, we retrieved the location of the planet from the synthetic cubes using $\discminer$ . We found strong consistency between the hydrodynamics codes, particularly in the density and velocity perturbations associated with planet-driven spirals. We also found a good agreement between the two radiative transfer codes: the disk temperature in $\mcfost$ and $\radmc$ models agrees within $\lesssim 3 \%$ everywhere in the domain. In synthetic $^{12}$ CO channel maps, this results in brightness temperature differences within $\pm1.5$ K in all our models. This good agreement ensures consistent retrieval of planet's radial/azimuthal location with only a few \% of scatter, with velocity perturbations varying $\lesssim 20 \%$ among the models. Notably, while the planet-opened gap is shallower in the $\Phantom$ simulation, we found that this does not impact the planet location retrieval. In summary, our results demonstrate that any combination of the tested hydrodynamics and radiative transfer codes can be used to reliably model and interpret planet-driven kinematic perturbations.

</div>

<div id="div_fig1">

<img src="tmp_2504.18643/././figures/dens2d_unstratified.png" alt="Fig1" width="100%"/>

**Figure 1. -** Perturbed density (normalized by the initial density), $\delta\rho/\rho_{\rm init}$ at (top panels) $Z/R=0.3$(3 scale heights above the midplane at $R=1$), (middle panels) $Z/R=0.1$(1 scale height above the midplane at $R=1$), and (bottom panels) midplane. From left to right, results with \texttt{FARGO3D}, \texttt{Idefix}, \texttt{Athena++}, \texttt{PLUTO}, and \texttt{Phantom}. At high altitudes, $\Phantom$ results appear noisy because of the small number of SPH particles in the low-density regions. (*fig:dens2d*)

</div>
<div id="div_fig2">

<img src="tmp_2504.18643/././figures/dens1d_phi_unstratified.png" alt="Fig10" width="100%"/>

**Figure 10. -** Azimuthal distribution of the density at (top panels) $R=1.3$ and (bottom panels) $R=0.7$. From left to right, each panel shows the density at the midplane, at $Z/R=0.1$, and $Z/R=0.3$, respectively. The scatter seen in the $\Phantom$ simulation is due to the finite number of particles used in the simulation, and is expected. (*fig:dens1d*)

</div>
<div id="div_fig3">

<img src="tmp_2504.18643/././figures/vtheta2d_unstratified.png" alt="Fig4" width="100%"/>

**Figure 4. -** Same as Figure \ref{fig:dens2d}, but for the meridional velocity $v_\theta$. (*fig:vtheta2d*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.18643"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# exoALMA I. Science Goals, Project Design and Data Products

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.18688-b31b1b.svg)](https://arxiv.org/abs/2504.18688)<mark>Appeared on: 2025-04-29</mark> -  _This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters_

</div>
<div id="authors">

R. Teague, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark>

</div>
<div id="abstract">

**Abstract:** Planet formation is a hugely dynamic process requiring the transport, concentration and assimilation of gas and dust to form the first planetesimals and cores. With access to extremely high spatial and spectral resolution observations at unprecedented sensitivities, it is now possible to probe the planet forming environment in detail. To this end, the exoALMA Large Program targeted fifteen large protoplanetary disks ranging between ${\sim}1\arcsec$ and ${\sim}7\arcsec$ in radius, and mapped the gas and dust distributions. $^{12}$ CO J=3-2, $^{13}$ CO J=3-2 and CS J=7-6 molecular emission was imaged at high angular ( ${\sim} 0\farcs15$ ) and spectral ( ${\sim} 100 {\rm m s^{-1}}$ ) resolution, achieving a surface brightness temperature sensitivity of ${\sim}1.5$ K over a single channel, while the 330 GHz continuum emission was imaged at 90 mas resolution and achieved a point source sensitivity of ${\sim} 40 \mu{\rm Jy beam^{-1}}$ . These observations constitute some of the deepest observations of protoplanetary disks to date. Extensive substructure was found in all but one disk, traced by both dust continuum and molecular line emission. In addition, the molecular emission allowed for the velocity structure of the disks to be mapped with excellent precision (uncertainties on the order of $10 {\rm m s^{-1}}$ ), revealing a variety of kinematic perturbations across all sources. From this sample it is clear that, when observed in detail, all disks appear to exhibit physical and dynamical substructure indicative of on-going dynamical processing due to young, embedded planets, large-scale, (magneto-)hydrodynamical instabilities or winds.

</div>

<div id="div_fig1">

<img src="tmp_2504.18688/figures/12CO_gallery.png" alt="Fig1" width="100%"/>

**Figure 1. -** A summary of the exoALMA sample with all sources showing evidence of gas substructures. All images show the $^{12}$CO (3-2) peak intensity obtained with \texttt{bettermoments} ([Teague and Foreman-Mackey 2018]())  using a quadratic fit to the data and masked with the CLEAN masks. Each source has a variable field of view and the open circle in the lower left of each image denotes the $0$\farcs$15$ synthesized beam for reference. The color maps is individually scaled for each source to emphasize the emission morphology rather than for a quantitative comparison (which can be found in  ([Galloway-Sprietsma, Bae and Izquierdo 2024]()) ). (*fig:CO_gallery_main*)

</div>
<div id="div_fig2">

<img src="tmp_2504.18688/figures/emission_surface_rotation_profile_demo.png" alt="Fig6" width="100%"/>

**Figure 6. -** The high angular and spectral resolution of the exoALMA data enable a detailed exploration of the 3D physical and dynamical structure of protoplanetary disks. Panel (a) shows the projection of the inferred emission surface of the $^{12}$CO emission from the disk around LkCa 15 shown in panel (b). As described in [Galloway-Sprietsma, Bae and Izquierdo (2024)](), the gray points are individual measurements while the black line shows a moving average with a window of $0$\farcs$15$. This deprojection allows for a precise $v_{\phi}$ profile to be extracted, as shown by the orange points in panel (c), which reveals a significant departure from purely Keplerian rotation, as shown by the dotted gray line  ([Stadler, Benisty and Winter 2024]()) . The global profile can be well recovered with a model, shown by the gray line, that includes a radial pressure gradient and the inclusion of the self-gravity of the disk  ([Longarini, et. al 2024]()) . The localized variations, most readily discernible in the residuals shown in panel (d), can be associated with localized pressure variations within the disk  ([Stadler, Benisty and Winter 2024]()) . (*fig:LkCa15_velocity*)

</div>
<div id="div_fig3">

<img src="tmp_2504.18688/figures/HD135344_gallery.png" alt="Fig3" width="100%"/>

**Figure 3. -** Example of the peak intensities derived from the `fiducial set' of images for HD 135344B. All panels share a $4$\arcsec$\! \times 4$\arcsec$$ field of view centered on the star. The spatial resolution of molecular emission images is $0$\farcs$15$ and the channel spacing is 100 ${\rm m s^{-1}}$ for both $^{12}$CO and $^{13}$CO and 200 ${\rm m s^{-1}}$ for CS. The continuum image has a synthesized beam of $90 {\rm mas} \times 76 {\rm mas}$ with a position angle of $85\fdg0$. (*fig:HD13344_gallery*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.18688"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cp}[1]{\textcolor{red}{CP: #1}}$</div>



<div id="title">

# exoALMA. X. channel maps reveal complex $^{12}$CO abundance distributions and a variety of kinematic structures with evidence for embedded planets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.18717-b31b1b.svg)](https://arxiv.org/abs/2504.18717)<mark>Appeared on: 2025-04-29</mark> -  _16 pages, 8 figures, accepted for publication in the ApJL exoALMA focus issue_

</div>
<div id="authors">

C. Pinte, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark>

</div>
<div id="abstract">

**Abstract:** We analyze the $^{12}$ CO $J=3-2$ data cubes of the disks in the exoALMA program.  13/15 disks reveal a variety of kinematic substructures in individual channels: large-scale arcs or spiral arms, localized velocity kinks, and/or multiple faints arcs that appear like filamentary structures on the disk surface.  We find kinematic signatures that are consistent with planet wakes in six disks: AA Tau, SY Cha, J1842, J1615, LkCa 15 and HD 143006. Comparison with hydrodynamical and radiative transfer simulations suggests planets with orbital radii between 80 and 310 au and masses between 1 and 5 M $_\mathrm{Jup}$ . Additional kinematic substructures limit our ability to place tight constraints on the planet masses. When the inclination is favorable to separate the upper and lower surfaces (near 45 $^\mathrm{o}$ , $*i.e.*$ in 7/15 disks), we always detect the vertical CO snowline and find that the $^{12}$ CO freeze-out is partial in the disk midplane, with a depletion factor of $\approx 10^{-3}$ -- $10^{-2}$ compared to the warm molecular layer. In these same seven disks, we also systematically detect evidence of CO desorption in the outer regions.

</div>

<div id="div_fig1">

<img src="tmp_2504.18717/./channels-1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Sources displaying large-scale non-Keplerian arcs. For each object, we show the channel at systemic velocity (central column), as well as channels at two velocity offsets on the blue side of the line (first two columns) and the opposite channels on the red side of the line (last two columns).  Solid lines indicate the expected location of the isovelocity curves at Keplerian velocities $\pm$100 m s. White arrows indicate deviations from the expected isovelocity curves for Keplerian rotation, and white circles highlight velocity kinks, *i.e.* distortions of the isovelocity curves. The beam is indicated as a gray circle and the horizontal white line is a $0$\farcs$2$ scale. The color map traces the square root of the intensity between zero and the peak. Note that the ringlike structure seen in CQ Tau is the continuum emission.  (*fig:arcs*)

</div>
<div id="div_fig2">

<img src="tmp_2504.18717/./channels-2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Sources with velocity kinks consistent with embedded planets (white circles). White arrows indicate additional deviations from Keplerian rotation. Beams and scales as in Fig \ref{fig:arcs}.  (*fig:kinks*)

</div>
<div id="div_fig3">

<img src="tmp_2504.18717/./cont_scatt.png" alt="Fig5" width="100%"/>

**Figure 5. -** $^{12}$CO $J=3-2$ observations from the Atacama Large Millimeter/submillimeter Array (ALMA), continuum emission, and near-infrared scattered light images for the five newly detected planet candidates. The first row shows the $^{12}$CO $J=3-2$ channels with the identified velocity kinks. The second row presents the corresponding peak intensity map. The third row displays a high-pass filtered peak intensity map using a Gaussian kernel twice the size of the beam, where we overlay the location of the planetary wake, derived using the same thermal structure as in the {\sf phantom} models and projected vertically onto the CO-emitting surface. The fourth row shows the exoALMA continuum  ([Curone, et. al 2025]()) , and the last row presents SPHERE DPI observations. The white circles indicate the velocity kinks shown in Fig. \ref{fig:kinks}, while the blue dot marks the deprojected location on the disk midplane. No SPHERE data are available for AA Tau, as it is currently too faint to lock the adaptive optics system.
 (*fig:cont*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.18717"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comment}[1]{\textit{\textcolor{red}{#1}}}$
$\newcommand{\refexoalma}[1]{\textbf{\textcolor{blue}{#1}}}$
$\newcommand{\revision}[1]{\textcolor{red}{#1}}$
$\newcommand{\galario}{\texttt{galario}\xspace}$
$\newcommand{\frank}{\texttt{frank}\xspace}$
$\newcommand{\discminer}{\texttt{discminer}\xspace}$</div>



<div id="title">

# exoALMA IV: Substructures, Asymmetries, and the Faint Outer Disk in Continuum Emission

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.18725-b31b1b.svg)](https://arxiv.org/abs/2504.18725)<mark>Appeared on: 2025-04-29</mark> -  _This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters_

</div>
<div id="authors">

P. Curone, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** The exoALMA Large Program targeted a sample of 15 disks to study gas dynamics within these systems, and these observations simultaneously produced continuum data at 0.9 mm (331.6 GHz) with exceptional surface brightness sensitivity at high angular resolution. To provide a robust characterization of the observed substructures, we performed a visibility space analysis of the continuum emission from the exoALMA data, characterizing axisymmetric substructures and nonaxisymmetric residuals obtained by subtracting an axisymmetric model from the observed data. We defined a nonaxisymmetry index and found that the most asymmetric disks predominantly show an inner cavity and consistently present higher values of mass accretion rate and near-infrared excess. This suggests a connection between outer disk dust substructures and inner disk properties. The depth of the data allowed us to describe the azimuthally averaged continuum emission in the outer disk, revealing that larger disks (both in dust and gas) in our sample tend to be gradually tapered compared to the sharper outer edge of more compact sources. Additionally, the data quality revealed peculiar features in various sources, such as shadows, inner disk offsets, tentative external substructures, and a possible dust cavity wall.

</div>

<div id="div_fig1">

<img src="tmp_2504.18725/./Figures/gallery_single_sources_maintext.png" alt="Fig17" width="100%"/>

**Figure 17. -** Comparison of data, $\frank$ model, CLEAN-imaged $\frank$ model, residuals, and polar plots for each disk (here showing AA Tau and CQ Tau and continued in Appendix \ref{sect:Appendix_Galleries}). (Top to bottom, left to right) First panel: fiducial continuum image of the observed data obtained with robust -0.5, with the synthesized beam’s FWHM shown as an ellipse in the lower left corner. The asinh function was applied to the color scale to visually enhance the fainter emission. Second panel: image of the frank model swept over $2\pi$ and reprojected, with normalized intensity and an asinh stretch. Each gap is marked by a dashed arc labeled “D" with its distance from the star in au, while solid arcs labeled “B"indicate the rings. The dotted line corresponds with the PA measured on gas data with $\discminer$ ([Izquierdo, Stadler and Bae 2025]())  and defines the $\phi=0$ angle of the polar plots, increasing counterclockwise (note possible PA differences with the one measured from $\galario$ and used in $\frank$, see Appendix \ref{sect:comparing_gas_dust_geom_param}). Third panel: $\frank$ model sampled at the same $uv$-points of the observation and imaged with CLEAN as the observed data. The color scale is the same as the data panel. Fourth panel: residuals obtained subtracting the \texttt{frank model} from the data. The residual visibilities were calculated at the same $uv$-points of the ALMA observations and imaged with CLEAN as the observed data. The color scale shows the residuals in units of the observed noise ($\sigma_\mathrm{rms}$). Rings and gaps are marked with solid and dashed ellipses, respectively. Fifth panel: polar plot of the data continuum image. Sixth panel: polar plot of the nonaxisymmetric residuals. The locations of rings and gaps are marked by solid and dashed horizontal lines, respectively. (*fig:gallery_single_sources_maintext*)

</div>
<div id="div_fig2">

<img src="tmp_2504.18725/./Figures/gallery_single_sources_appendix1.png" alt="Fig2" width="100%"/>

**Figure 2. -** Same as Fig. \ref{fig:gallery_single_sources_maintext} but for DM Tau and HD 135344B. (*fig:gallery_single_sources_appendix1*)

</div>
<div id="div_fig3">

<img src="tmp_2504.18725/./Figures/gallery_single_sources_appendix2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Same as Fig. \ref{fig:gallery_single_sources_maintext} but for HD 143006 and HD 34282. (*fig:gallery_single_sources_appendix2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.18725"></div>

...

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\discminer}{\textsc{discminer}}$
$\newcommand{\disksurf}{\texttt{disksurf}}$
$\newcommand{\gofish}{\texttt{GoFish}}$
$\newcommand{\zr}{\langle z / r \rangle}$
$\newcommand{\linmix}{\texttt{linmix}}$
$\newcommand$
$\newcommand{\maria}[1]{#1}$
$\newcommand{\mariatwo}[1]{#1}$
$\newcommand{\newcite}[1]{\textcolor{red}{\textbf{#1}}}$</div>



<div id="title">

# exoALMA V: Gaseous Emission Surfaces and Temperature Structures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.19902-b31b1b.svg)](https://arxiv.org/abs/2504.19902)<mark>Appeared on: 2025-04-29</mark> -  _This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters_

</div>
<div id="authors">

M. Galloway-Sprietsma, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** Analysis of the gaseous component in protoplanetary disks can inform us about their thermal and physical structure, chemical composition, and kinematic properties, all of which are crucial for understanding various processes within the disks.By exploiting the asymmetry of the line emission, or via line profile analysis, we can locate the emitting surfaces.Here, we present the emission surfaces of the exoALMA sources in $^{12}$ CO $J=3-2$ , $^{13}$ CO $J=3-2$ , and CS $J=7-6$ . We find that $^{12}$ CO traces the upper disk atmosphere, with mean $\zr$ values of $\approx$ 0.28, while $^{13}$ CO and CS trace lower regions of the disk with mean $\zr$ values of $\approx$ 0.16 and $\approx$ 0.18, respectively.We find that $^{12}$ CO $\zr$ and the disk mass are positively correlated with each other; this relationship offers a straightforward way to infer the disk mass.We derive 2-D $r-z$ temperature distributions of the disks. Additionally, we search for substructure in the surfaces and radial intensity profiles; we find evidence of localized substructure in the emission surfaces and peak intensity profiles of nearly every disk, with this substructure often being co-incident between molecular tracers, intensity profiles, and kinematic perturbations. Four disks display evidence of potential photo-desorption, implying that this effect may be common even in low FUV star-forming regions. For most disks, we find that the physical and thermal structure is more complex than analytical models can account for, highlighting a need for more theoretical work and a better understanding of the role of projection effects on our observations.

</div>

<div id="div_fig1">

<img src="tmp_2504.19902/./exoALMA_emissionsurfaces_12co.png" alt="Fig11" width="100%"/>

**Figure 11. -** Non-parametric (data points) and parametric surfaces (solid and dashed curves) for $^{12}$CO $J=3-2$. The larger black points represent radially binned data. We exclude two disks $\maria${(HD 135344B and J1604)} whose surfaces could not be extracted due to a face-on geometry. Background gray lines show scale heights $(z/r)$ from 0.1 to 0.6 with 0.1 increments. The darker blue solid line shows the tapered power law surface fit from $\discminer$ , and the light blue and gray dashed line shows the tapered power law fit from $\disksurf$ , which uses the background data points. The gray cross markers for AA Tau and SY Cha show the diffuse outer surface points that we do not include in the parametric fits. (*fig:surfaces_12co*)

</div>
<div id="div_fig2">

<img src="tmp_2504.19902/./exoALMA_all_rolling_surfaces.png" alt="Fig14" width="100%"/>

**Figure 14. -** Rolling surfaces made from the raw $r-z$\disksurf  emission surface points for $^{12}$CO $J=3-2$(blue), $^{13}$CO $J=3-2$(orange), and CS $J=7-6$(pink). Vertical dashed lines denote emission surface substructure, whose identification is discussed in Section \ref{sec:substucture}. We label dips in the emission surfaces with the nomenclature first used in [Law, Teague and Loomis (2021)](), where `Z' denotes a dip in the emission surface, followed by the radial location in au. We also plot the continuum rings (solid gray lines) and gaps (dashed gray lines) identified in [Curone, et. al (2025)]().
    Thin background gray lines show scale heights $(z/r)$ from 0.1 to 0.6 with 0.1 increments. The beamsize of $0$\farcs$15$ is shown in the upper right corner for each disk. AA Tau is zoomed in to better show structure. (*fig:rolling_surfaces*)

</div>
<div id="div_fig3">

<img src="tmp_2504.19902/./exoALMA_emissionsurfaces_13co.png" alt="Fig12" width="100%"/>

**Figure 12. -** The same as Figure \ref{fig:surfaces_12co}, but for $^{13}$CO $J=3-2$. The orange solid line shows the tapered power law surface fit from $\discminer$ , and the gray and peach dashed line shows the tapered power law fit from $\disksurf$ . (*fig:surfaces_13co*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.19902"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mj}{M_{\rm Jup}}$
$\newcommand{\phip}{\phi_{p}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\discminer}{\textsc{discminer}}$
$\newcommand{\disksurf}{\textsc{disksurf}}$
$\newcommand{\mcfost}{\textsc{mcfost}}$
$\newcommand{\radmc}{\textsc{radmc3d}}$
$\newcommand{\exoalma}{exoALMA}$
$\newcommand{\filfinder}{\textsc{FilFinder}}$
$\newcommand{\twCOfull}{^{12}CO J=3-2}$
$\newcommand{\twCO}{^{12}CO}$
$\newcommand{\thCO}{^{13}CO}$
$\newcommand{\thCOfull}{^{13}CO J=3-2}$
$\newcommand{\eiCO}{C^{18}O}$
$\newcommand{\eiCOfull}{C^{18}O J=3-2}$
$\newcommand{\CSfull}{CS J=7-6}$
$\newcommand{\aatau}{AA Tau}$
$\newcommand{\cqtau}{CQ Tau}$
$\newcommand{\dmtau}{DM Tau}$
$\newcommand{\hdthree}{HD 34282}$
$\newcommand{\hdone}{HD 135344B}$
$\newcommand{\hdonefour}{HD 143006}$
$\newcommand{\jfour}{J1604}$
$\newcommand{\jfifteen}{J1615}$
$\newcommand{\jfortytwo}{J1842}$
$\newcommand{\jfiftytwo}{J1852}$
$\newcommand{\lkca}{LkCa 15}$
$\newcommand{\mwcsev}{MWC 758}$
$\newcommand{\pdssix}{PDS 66}$
$\newcommand{\sycha}{SY Cha}$
$\newcommand{\vforty}{V4046 Sgr}$
$\newcommand{\arraystretch}{0.9}$
$\newcommand{\arraystretch}{0.95}$
$\newcommand{\arraystretch}{0.95}$</div>



<div id="title">

# exoALMA III: Line-intensity Modeling and System Property Extraction from Protoplanetary Disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.19986-b31b1b.svg)](https://arxiv.org/abs/2504.19986)<mark>Appeared on: 2025-04-29</mark> -  _This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters_

</div>
<div id="authors">

A. F. Izquierdo, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark>

</div>
<div id="abstract">

**Abstract:** The ALMA large program $\exoalma$ offers a unique window into the three-dimensional physical and dynamical properties of 15 circumstellar discs where planets may be actively forming. Here, we present an analysis methodology to map the gas disc structure and substructure encoded in $\twCO$ , $\thCO$ , and CS line emission from our targets. To model and characterize the disc structure probed by optically thin species, such as CS and, in some cases, $\thCO$ , we introduce a composite line profile kernel that accounts for increased intensities caused by the projected overlap between the disc's front and back side emission.Our workflow, built on the $\discminer$ modelling framework, incorporates an improved iterative two-component fitting method for inclined sources ( $i>40^\circ$ ), to mitigate the impact of the disc backside on the extraction of velocity maps. Also, we report best-fit parameters for the Keplerian stellar masses, as well as inclinations, position angles, systemic velocities, rotation direction, and emission surfaces of the discs in our sample.

</div>

<div id="div_fig1">

<img src="tmp_2504.19986/./figures/hd135344/channel_maps_hd135344_allmol.png" alt="Fig11" width="100%"/>

**Figure 11. -** Selected $\twCO$(top), $\thCO$(middle), and CS (bottom) intensity channels for the disc of $\hdone$ . Each panel displays, from top to bottom, the channel maps for the data, the best-fit model, and the corresponding residuals, with residuals below three times the rms noise whited out. Peak intensity maps and residuals are shown in the leftmost column of each panel. Solid and dashed lines overlaid on the channel maps represent isovelocity contours from the model's front and back sides.
               (*fig:channel_maps_hd135344*)

</div>
<div id="div_fig2">

<img src="tmp_2504.19986/./figures/lkca15/channel_maps_lkca15_allmol.png" alt="Fig12" width="100%"/>

**Figure 12. -** Same as Figure \ref{fig:channel_maps_hd135344}, but for the disc of $\lkca$ . The $\thCO$  and CS models are assumed to be optically thin and therefore use a `sum' kernel to combine the contributions of the front and back sides to the total intensity (see Sects. \ref{sec:models} and \ref{sec:single_moments} for details). CS isovelocity contours are shown in yellow to highlight regions where increased intensity results from overlapping column densities of the front and back sides at matching line-of-sight velocities.
               (*fig:channel_maps_lkca15*)

</div>
<div id="div_fig3">

<img src="tmp_2504.19986/./figures/flowchart_data_vs_model_v0p3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Flowchart illustrating the extraction of line profile properties for $\twCOfull$  emission from the disc of $\lkca$ .
               (*fig:flowchart*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.19986"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\gr}[1]{\textbf{\color{cyan}GR: #1}}$
$\newcommand{\cl}[1]{\textbf{\color{green} #1}}$
$\newcommand{\discminer}{\textsc{discminer} }$
$\newcommand{\disksurf}{\texttt{disksurf} }$</div>



<div id="title">

# exoALMA XV: Interpreting the height of CO emission layer

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.20012-b31b1b.svg)](https://arxiv.org/abs/2504.20012)<mark>Appeared on: 2025-04-29</mark> -  _23 pages, 12 figures. This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters available at this https URL_

</div>
<div id="authors">

G. P. Rosotti, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** The availability of exquisite data and the development of new analysis techniques have enabled the study of emitting heights in proto-planetary disks. In this paper we introduce a simple model linking the emitting height of CO to the disk surface density and temperature structure. We then apply the model to measurements of the emitting height and disk temperature conducted as part of exoALMA, integrated with additional legacy measurements from the MAPS Large Programme, to derive CO column densities and surface density profiles (assuming a CO abundance) for a total of 14 disks. A unique feature of the method we introduce to measure surface densities is that it can be applied to optically thick observations, rather than optically thin as conventionally done. While we use our method on a sample of well studied disks where temperature structures have been derived using two emission lines, we show that reasonably accurate estimates can be obtained also when only one molecular transition is available. With our method we obtain independent constraints from $^{12}$ CO and $^{13}$ CO and we find they are in general good agreement using the standard $^{12}$ C/ $^{13}$ C isotopic ratio. The masses derived from our method are systematically lower compared with the values derived dynamically from the rotation curve if using an ISM CO abundance, implying that CO is depleted by a median factor $\sim$ 20 with respect to the ISM value, in line with other works that find that CO is depleted in proto-planetary disks.

</div>

<div id="div_fig1">

<img src="tmp_2504.20012/./Nph_ttauri.png" alt="Fig9" width="100%"/>

**Figure 9. -** Left panel: Emitting height as a function of radius for $^{12}$CO, assuming a disk mass of 0.01 $M_\odot$. We show the values from the DALI thermo-chemical model, as well as from our semi-analytical model, for different values of the free parameter $N_\mathrm{ph}$. Central panel: Emitting height as a function of radius for $^{13}$CO, assuming a disk mass of 0.01 $M_\odot$ for different values of $N_{ph}$. We compare the DALI data (black line) with the emitting layer predicted assuming $N_{ph,13}=N_{ph,12}$(blue line) and $N_{ph,13}=N_{ph,12}$$   ^{13}\mathrm{C}/^{12}\mathrm{C}$(orange line). Right panel: Emitting height as a function of radius for $^{12}$CO, assuming a disk mass of 0.01 $M_\odot$ orbiting a Herbig star. We show the values from the DALI thermo-chemical model, as well as from our semi-analytical model, for different values of the free parameter $N_\mathrm{ph}$. (*fig:comparison_dali*)

</div>
<div id="div_fig2">

<img src="tmp_2504.20012/./lkca15_sigma_comp_DM_single.png" alt="Fig2" width="100%"/>

**Figure 2. -** Surface density as a function of radius derived with the stratified model for LkCa 15. We show the results for both $^{12}$CO and $^{13}$CO; shaded regions show the uncertainties (see text for a discussion of the radial dependence of the uncertainty). For comparison we also show the surface density implied by the fit to the rotation curve presented in this paper series in [Longarini, Lodato and Rosotti (2025)]() and a constraint from optically thin C$^{18}$O and C$^{17}$O lines presented in [Sturm, et. al (2023)](). (*fig:results_lkca15*)

</div>
<div id="div_fig3">

<img src="tmp_2504.20012/./critical_column.png" alt="Fig1" width="100%"/>

**Figure 1. -** $N_\tau$, the CO column to have $\tau=2/3$ for the J=3-2 and J=2-1 transition of $^{12}$CO and $^{13}$CO, as a function of temperature. The photo-dissociation column value we show is only a reference and it will be estimated more precisely in section \ref{sec:calibration}. (*fig:critical_column*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.20012"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\farcsec}{\mbox{\ensuremath{\hspace{1pt}.\!\!^{\prime\prime}}}}$
$\newcommand{\vdag}{(v)^\dagger}$</div>



<div id="title">

# exoALMA XVII. Characterizing the Gas Dynamics around Dust Asymmetries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.20023-b31b1b.svg)](https://arxiv.org/abs/2504.20023)<mark>Appeared on: 2025-04-29</mark> -  _19 pages, 8 figures, 1 table, This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters_

</div>
<div id="authors">

L. Wölfer, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The key planet-formation processes in protoplanetary disks remain an active matter of research. One promising mechanism to radially and azimuthally trap millimeter-emitting dust grains, enabling them to concentrate and grow into planetesimals, is anticyclonic vortices. While dust observations have revealed crescent structures in several disks, observations of their kinematic signatures are still lacking. Studying the gas dynamics is, however, essential to confirm the presence of a vortex and understand its dust trapping properties. In this work, we make use of the high-resolution and sensitivity observations conducted by the exoALMA large program to search for such signatures in the $^{12}$ CO and $^{13}$ CO  molecular line emission of four disks with azimuthal dust asymmetries: HD 135344B, HD 143006, HD 34282, and MWC 758. To assess the vortex features, we constructed an analytical vortex model and performed hydrodynamical simulations. For the latter, we assumed two scenarios: a vortex triggered at the edge of a dead zone and of a gap created by a massive embedded planet. These models reveal a complex kinematical morphology of the vortex. When compared to the data, we find that none of the sources show a distinctive vortex signature around the dust crescents in the kinematics.HD 135344B exhibits a prominent feature similar to the predictions from the simulations, thus making this the most promising target for sensitive follow-up studies at higher resolution and in particular with less abundant molecules at higher resolution and sensitivity, to trace closer to the disk midplane.

</div>

<div id="div_fig1">

<img src="tmp_2504.20023/./plots/Vortex_analytical_model_arcsec.png" alt="Fig3" width="100%"/>

**Figure 3. -** Analytical model of a vortex, described as elliptic streamlines of constant velocity with a Gaussian velocity profile. The first two panels show the absolute velocity of the vortex in both the vortex frame and sky frame. The last panel depicts the velocity projected along the line-of-sight, showing a complex morphology purely resulting from the projection. In the last two panels, the underlying disk geometry is overlaid as grey contours, corresponding to a position angle of $243 ◦ee$ and inclination of $16 ◦ee$(same as for HD 135344B). (*fig:vortexAnalytical*)

</div>
<div id="div_fig2">

<img src="tmp_2504.20023/./plots/cont_and_scat_and_12CO_v1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Overview of the four sources studied in this work, shown on the same spatial scale for the dust (top two rows) and gas (bottom row). Top: Millimetre dust continuum as observed within the exoALMA program \citep{Curone_exoALMA}. Middle: Micron-sized dust observed through near-infrared scattered light with SPHERE \citep{Stolker2017,Boer2021,Ren2023}. The SPHERE images are normalized to the peak value and shown with a logarithmic normalization of the color map to emphasize the structures in the outer disk. To highlight the location of the crescents, two continuum contours around the peak of the dust asymmetries are overlaid on the dust images. Bottom: Peak intensity of the $^{12}$CO $J=3-2$ emission line with overlaid continuum emission (five contour levels equally placed between 3$\sigma$ and the peak flux). Note the larger spatial scale of the gas versus dust images. (*fig:dust_gas*)

</div>
<div id="div_fig3">

<img src="tmp_2504.20023/./plots/vel_res_overview_arcsec.png" alt="Fig2" width="100%"/>

**Figure 2. -** Velocity residuals $\Delta v_0$ after subtraction of the Keplerian model from the data, shown for $^{12}$CO and $^{13}$CO $J=3-2$ and all four targets. For HD 34282, only emission from the top side of the disk was considered when making the $v_0$ map, following the procedure described in \cite{Izquierdo_exoALMA}. The beam is shown in the bottom left corner of the top row. The red-blue circle and arrow indicate the underlying rotation pattern of each disk. To highlight the location of the crescents, two continuum contours around the dust asymmetries are overlaid on the images. (*fig:VelRes*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.20023"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# exoALMA. VI. Rotating under Pressure: \ $\vspace{0.1cm}$     \text{Rotation curves, azimuthal velocity substructures, and pressure variations}

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.20036-b31b1b.svg)](https://arxiv.org/abs/2504.20036)<mark>Appeared on: 2025-04-29</mark> -  _This paper is part of the exoALMA Focus Issue of The Astrophysical Journal Letters. 37 pages, 19 figures_

</div>
<div id="authors">

J. Stadler, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>, <mark>M. Flock</mark>, <mark>I. Hammond</mark>

</div>
<div id="abstract">

**Abstract:** The bulk motion of the gas in protoplanetary disks around newborn stars is nearly Keplerian. By leveraging the high angular and spectral resolution of ALMA, we can detect small-scale velocity perturbations in molecular line observations caused by local gas pressure variations in the disk, possibly induced by embedded protoplanets.This paper presents the azimuthally averaged rotational velocity and its deviations from Keplerian rotation ( $\dvphi$ ) for the exoALMA sample, as measured in the $\twCOfull$ and $\thCOfull$ emission lines. The rotation signatures show evidence for vertically stratified disks, in which $\thCO$ rotates faster than $\twCO$ due to a distinct thermal gas pressure gradient at their emitting heights. We find $\dvphi$ -substructures in the sample on both small ( $\sim$ 10 au) and large ( $\sim$ 100 au) radial scales, reaching deviations up to 15 \% from background Keplerian velocity in the most extreme cases.More than 75 \% of the rings and 80 \% of the gaps in the dust continuum emission resolved in $\dvphi$ are co-located with gas pressure maxima and minima, respectively. Additionally, gas pressure substructures are observed far beyond the dust continuum emission. For the first time, we determined the gas pressure derivative at the midplane from observations and found it to align well with the dust substructures within the given uncertainties. Based on our findings, we conclude that gas pressure variations are likely the dominant mechanism for ring and gap formation in the dust continuum.

</div>

<div id="div_fig1">

<img src="tmp_2504.20036/./figures/paper_vphi_dpdr_symgap_comparison_2row.png" alt="Fig1" width="100%"/>

**Figure 1. -** (a) Azimuthal velocities (left axis, black lines) and midplane pressure (right axis, blue lines), (b) along with the deviations from Keplerian rotation and the first radial derivative of the pressure, for a hydrodynamic model with an imposed Gaussian gap in the pressure at $R=240 $au. The pressure minimum and maximum locations are marked with vertical purple dashed and full lines, respectively. The circles in (b) mark the sign of the radial derivative of \dvphi  coinciding with the pressure minimum ($\oplus$) and maximum ($\ominus$), respectively. (*fig:vphi_press*)

</div>
<div id="div_fig2">

<img src="tmp_2504.20036/./figures/dvphi_percent_all_disks_12co_marker_largescale_contgrad_fullpage_R90_nonsmooth.png" alt="Fig6" width="100%"/>

**Figure 6. -** Radial profiles of \dvphi  for \twCOfull   of all sources focused on the region beyond the continuum substructures using the High Surface Brightness Sensitivity Images. The error bars show the standard deviation within each extracted radial annulus. The vertical dashed-dotted line indicates the radius that encompasses 95\% of the continuum emission, with the signs of the radial derivatives of \dvphi  marked at this location. The beam size is shown in the lower left corner. The \thCO\dvphi  radial profiles can be found in Fig. \ref{fig:dvphi_large_13co} of the Appendix. (*fig:dvphi_large*)

</div>
<div id="div_fig3">

<img src="tmp_2504.20036/./figures/dvphi_percent_all_disks_12co_marker_smallscale_promcont_contgrad_fullpage_R90_nonsmooth.png" alt="Fig7" width="100%"/>

**Figure 7. -** Radial profiles of \dvphi  for \twCOfull   of all sources focused on the region of the continuum substructures, using the High Resolution Images. The profiles are plotted starting at two beam sizes from the disk center and the error bars show the standard deviation of each bin. The gray background gradient highlights the \texttt{frank} radial profiles of the dust continuum emission normalized to its peak. The locations of continuum rings, gaps, and crescents are plotted in solid, dashed, and dotted vertical gray lines, respectively \citep[][]{Curone_exoALMA}. We expect dust rings and gaps to be co-located with a radially decreasing and increasing \dvphi  profile, if caused by pressure maxima and minima, respectively. The beam size is plotted in the lower right corner. The \thCO\dvphi  radial profiles can be found in Fig. \ref{fig:dvphi_small_13co} of the Appendix. (*fig:dvphi_small*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.20036"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

471  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

23  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

21  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
